In [8]:
from coding import search_w

import numpy as np
from scipy import fftpack as FT

# Let's debug Cyclic Code a bit

## n = 7, s = 2, d = 10

In [9]:
# global variables:
n = 7
s = 2
d = 10
#d = 1

In [10]:
np.set_printoptions(precision=4,linewidth=200.0)
W, fake_W, W_perp, S, C_1 = search_w(n, s)

In [11]:
def estimator_generator(n, s):
    estimator = np.zeros((n, s+1), dtype=complex)
    #z_gen_func = np.vectorize(lambda t: np.exp(-2*np.pi*t*1j/n))
    z_gen_func = np.vectorize(lambda t: np.exp(2*np.pi*t*1j/n))
    col1 = z_gen_func(np.arange(n))
    for i in range(s+1):
        estimator[:, i] = np.power(col1, i)
    return estimator

p_estimator = estimator_generator(n, s)
print(p_estimator)

[[ 1.0000+0.j      1.0000+0.j      1.0000+0.j    ]
 [ 1.0000+0.j      0.6235+0.7818j -0.2225+0.9749j]
 [ 1.0000+0.j     -0.2225+0.9749j -0.9010-0.4339j]
 [ 1.0000+0.j     -0.9010+0.4339j  0.6235-0.7818j]
 [ 1.0000+0.j     -0.9010-0.4339j  0.6235+0.7818j]
 [ 1.0000+0.j     -0.2225-0.9749j -0.9010+0.4339j]
 [ 1.0000+0.j      0.6235-0.7818j -0.2225-0.9749j]]


## simulate epsilon here (assume worker 0 1 fails):

In [12]:
simulated_epsilon = np.zeros((n, d), dtype=complex)
simulated_epsilon[0:2,:] = -100.0+0.0j
print(simulated_epsilon)

[[-100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j]
 [-100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j -100.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]]


## obtain E_2 (simulated) here:

In [13]:
simulated_E2 = np.dot(W_perp, simulated_epsilon)
print(simulated_E2)

[[ -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j  -3.7430-16.3993j]
 [ -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j  -3.7430+16.3993j]
 [-29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j -29.3859+36.8488j]
 [-61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j -61.3621+29.5505j]]


## calculated a to get the full E matrix

In [14]:
import copy
poly_a = np.zeros(s+1, dtype=complex)
poly_a[-1] = 1+0j
fake_poly_a = copy.deepcopy(poly_a)
# calculate a over here:
E_2 = simulated_E2

def _cls_solver(A, b):
    return np.dot(np.dot(np.linalg.inv(np.dot(_array_getH(A), A)), _array_getH(A)),b)

def _array_getH(ndarray):
    # get conjugate transpose of a np.ndarray
    return ndarray.conj().T

_shape = E_2.shape
_s = _shape[0]/2
_d = _shape[1]
_X = np.take(E_2, np.array([range(-i-(_s+1), -i-2+1) for i in range(_s)]).reshape(-1), axis=0).reshape((_s, _s*_d), order='F')
# we use tmp_y as the start point to obtain the full E matrix
tmp_y = np.take(E_2, np.array([-i-1 for i in range(_s)]), axis=0)
_y = tmp_y.reshape(-1, order='F')
alpha_normal = _cls_solver(np.transpose(_X), _y.reshape(_y.shape[0], 1))
poly_a[0:s] = alpha_normal.reshape(-1)
fake_poly_a[0:s] = -alpha_normal.reshape(-1)

# check matrix formula and calculated result:
print("Formulated X is :")
print(np.transpose(_X))
print
print("Formulated y is :")
print(_y.reshape(_y.shape[0], 1))
print
print("E_2 is: ")
print(E_2)
print
print("Polynomial a is: ")
print(poly_a)

Formulated X is :
[[ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]
 [ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]]

Formulated y is :
[[-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]
 [-29.3859+36.8488j]
 [-61.3621+29.

In [15]:
# function to obtain matrix E
def obtain_E(alpha, E_2, s):
    # obtain E_1 in shape of n-2s by d
    _processing_y = np.transpose(E_2)[:, -s:]

    for i in range(n-2*s):
        tmp = np.dot(_processing_y[:,-s:], alpha)
        print("Obtaining E[{}]".format(i))
        print(_processing_y[:,-s:])
        print
        print("Calculated Results:")
        print(tmp)
        print
        _processing_y = np.concatenate((_processing_y, tmp), axis=1)
        #print(_processing_y[0,-s:])
        #print('============================================================')
    #print(_processing_y[0, s:])
    #print("----------------------------------------------------------------")
    #exit()
    return np.transpose(_processing_y[:, s:])

# function to obtain epsilon
def obtain_epsilon(E):
    return FT.ifft(E, n=n, axis=0)

# Test New Method

In [19]:
from numpy.polynomial import polynomial as P
#q1 = np.exp(-2*np.pi*0*1j/n)
#q2 = np.exp(-2*np.pi*1*1j/n)
q1 = np.exp(2*np.pi*0*1j/n)
q2 = np.exp(2*np.pi*1*1j/n)
roots = [q1, q2]
np_poly=P.polyfromroots(roots)
print("Poly solved by Numpy: ")
print(np_poly)
print
q1_conj = np.conj(q1)
q2_conj = np.conj(q2)
print("q_1 is {}".format(q1))
print("q_2 is {}".format(q2))
print("q_1_conj is {}".format(q1_conj))
print("q_2_conj is {}".format(q2_conj))
bbb = - (q1 + q2)
ccc = q1 * q2
#bbb = - (q1_conj + q2_conj)
#ccc = q1_conj * q2_conj
print(bbb)
print(ccc)
print
real_poly=[ccc, bbb, 1]
print("The Correct polynomial should be: {}".format(real_poly))
print("The one we get is: {}".format(poly_a))
print
print(np.dot(p_estimator, real_poly))

Poly solved by Numpy: 
[ 0.6235+0.7818j -1.6235-0.7818j  1.0000+0.j    ]

q_1 is (1+0j)
q_2 is (0.623489801859+0.781831482468j)
q_1_conj is (1-0j)
q_2_conj is (0.623489801859-0.781831482468j)
(-1.62348980186-0.781831482468j)
(0.623489801859+0.781831482468j)

The Correct polynomial should be: [(0.62348980185873359+0.7818314824680298j), (-1.6234898018587336-0.7818314824680298j), 1]
The one we get is: [-0.6235-0.7818j  1.6235+0.7818j  1.0000+0.j    ]

[ 0.0000 +0.0000e+00j  0.0000 +1.1102e-16j  0.8460 -1.0609e+00j  3.0489 -4.4409e-16j  2.3705 +2.9725e+00j -0.6784 +2.9725e+00j -1.2225 +5.8874e-01j]


In [38]:
_row_vec = np.zeros((1, n-2*s))
_row_vec[0][-1]=1
_recover_final=np.zeros((1,n),dtype=complex)
#print(np.dot(p_estimator, poly_a))
# if we let a = -a
#print
#print(np.dot(p_estimator, fake_poly_a))
print("="*80)
estimation = np.dot(p_estimator, fake_poly_a)
print("Step 0: The results we test the zeros points: ")
print(estimation)
print("="*80)
print
err_indices = [i for i, elem in enumerate(estimation) if (elem.real > 1e-10 or elem.imag > 1e-10)]
print("="*80)
print("Step 1: C_1 matrix we have: ")
print(C_1)
print("="*80)
print
recover=C_1.take(err_indices, axis=0).take(np.arange(n-2*s),axis=0)
print("="*80)
print("Step 2: (n-2s)by(n-2s) matrix we get: ")
print(recover)
print("="*80)
print
remaining_indices = err_indices[0:n-2*s]

inv_recover=np.linalg.inv(recover)
print("="*80)
print("Step 3: inv of the (n-2s)by(n-2s) matrix: ")
print(inv_recover)
print("check result: ")
print(np.dot(inv_recover, recover))
print("="*80)
print

row_recover = np.dot(_row_vec, inv_recover)
print("="*80)
print("Step 4: [0, ..., 1] vec we use: ")
print(_row_vec)
print("dot product it with inv :")
print(row_recover)
print("="*80)
print


_recover_final[0][[remaining_indices]] = row_recover[0]
print("="*80)
print("Step 5: insert zeros to appropriate positions ")
print(_recover_final)
print("="*80)
print
#decoded_grad = np.dot(_recover_final, R)

Step 0: The results we test the zeros points: 
[ -1.3323e-15 -1.4433e-15j  -1.1657e-15 -1.2212e-15j   8.4601e-01 -1.0609e+00j   3.0489e+00 -1.5543e-15j   2.3705e+00 +2.9725e+00j  -6.7845e-01 +2.9725e+00j  -1.2225e+00 +5.8874e-01j]

Step 1: C_1 matrix we have: 
[[ 0.3780+0.j      0.3780+0.j      0.3780+0.j    ]
 [ 0.3780+0.j      0.2357-0.2955j -0.0841-0.3685j]
 [ 0.3780+0.j     -0.0841-0.3685j -0.3405+0.164j ]
 [ 0.3780+0.j     -0.3405-0.164j   0.2357+0.2955j]
 [ 0.3780+0.j     -0.3405+0.164j   0.2357-0.2955j]
 [ 0.3780+0.j     -0.0841+0.3685j -0.3405-0.164j ]
 [ 0.3780+0.j      0.2357+0.2955j -0.0841+0.3685j]]

Step 2: (n-2s)by(n-2s) matrix we get: 
[[ 0.3780+0.j     -0.0841-0.3685j -0.3405+0.164j ]
 [ 0.3780+0.j     -0.3405-0.164j   0.2357+0.2955j]
 [ 0.3780+0.j     -0.3405+0.164j   0.2357-0.2955j]]

Step 3: inv of the (n-2s)by(n-2s) matrix: 
[[-0.4339 +1.9010e+00j  3.5135 -1.1749e-15j -0.4339 -1.9010e+00j]
 [-0.7818 +3.4254e+00j  3.9474 -1.9010e+00j -3.1656 -1.5245e+00j]
 [-0.4339 +

True 1 traspose * G: <br />
[[ 0.0006 +0.0000e+00j  0.0006 +0.0000e+00j  0.0006 +0.0000e+00j ...,  0.0007 +6.7763e-21j  0.0007 +6.7763e-21j  0.0007 +6.7763e-21j]]

The results we get:  <br />
[ -2.8031e-04 -1.8633e-05j  -2.8031e-04 -1.8633e-05j  -2.8031e-04 -1.8633e-05j ...,  -4.5684e-05 -1.5724e-04j  -4.5684e-05 -1.5724e-04j  -4.5684e-05 -1.5724e-04j]

## Full E matrix calculated

In [26]:
# now let's find the whole E and also epsilon
E_1=obtain_E(alpha, E_2, s)
# concatenate E_1 and E_2 to obtain E
E = np.concatenate((E_1, E_2), axis=0)

epsilon = obtain_epsilon(E)

#print(epsilon)

Obtaining E[0]
[[-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]]

Calculated Results:
[[-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]]

Obtaining E[1]
[[-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j 

## finally let's look at the epsilon matrix we get

In [27]:
print(epsilon)

[[ -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j
   -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j]
 [  4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j
    4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j]
 [  3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j
    3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j]
 [ -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j
   -5.1483e-14 +2.080

## we if we give it a factor? say sqrt(n)

In [28]:
print(np.sqrt(n)*epsilon)

[[ -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j
   -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j]
 [  1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j
    1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j]
 [  8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j
    8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j]
 [ -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j
   -1.3621e-13 +5.503

In [29]:
# let's try to fix it
tmp = np.sqrt(n)*epsilon
a = np.roll(tmp, s-1, axis=0)
print(a)

[[ -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j
   -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j]
 [ -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j
   -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j]
 [  1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j
    1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j]
 [  8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j
    8.7618e-14 +1.182

In [20]:
print("n=45, s=5")

n=45, s=5


Polynomial solved from numpy:
[-0.1736-0.9848j  2.1494+4.4068j -6.4977-7.2164j  8.2351+5.1459j -4.7131-1.3515j  1.0000+0.j    ]

Our Polynomial: 
[-0.4320-0.4451j  2.5277+2.1336j -6.1050-3.8598j  7.3693+3.0851j -4.3599-0.9137j  1.0000+0.j    ]


Polynomial solved from numpy:
[-0.1736-0.9848j  2.1494+4.4068j -6.4977-7.2164j  8.2351+5.1459j -4.7131-1.3515j  1.0000+0.j    ]

Our Polynomial: 
[-0.2295+1.8293j -0.6282-6.0499j  1.6318+6.3744j  0.6011-1.9989j -2.3748-0.1552j  1.0000+0.j    ]


In [22]:
print("n=45, s=4")

n=45, s=4


Polynomial solved from numpy:
[ 0.6691+0.7431j -3.1968-2.3226j  5.3927+2.401j  -3.8651-0.8215j  1.0000+0.j    ]

Our Polynomial: 
[ 0.6693+0.7437j -3.1977-2.3242j  5.3939+2.4023j -3.8656-0.8219j  1.0000+0.j    ]


In [21]:
print("n=45, s=3")

n=45, s=3


Polynomial solved from numpy:
[-0.9135-0.4067j  2.8651+0.8215j -2.9515-0.4148j  1.0000+0.j    ]

Our Polynomial: 
[-0.9135-0.4068j  2.8651+0.8216j -2.9515-0.4148j  1.0000+0.j    ]


In [23]:
print("n=45, s=2")

n=45, s=2


Polynomial solved from numpy:
[ 0.9903+0.1392j -1.9903-0.1392j  1.0000+0.j    ]

Our Polynomial: 
[ 0.9903+0.1392j -1.9903-0.1392j  1.0000+0.j    ]
